In [21]:
import numpy as np
import sys; sys.path.append("../")
import tiktoken

import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

In [4]:
train_data = np.memmap('../data/openwebtext/train.bin', dtype=np.uint16, mode='r')

In [5]:
print(train_data.shape)

(9035582489,)


In [6]:
print(train_data[0])

8585


In [8]:
enc = tiktoken.get_encoding("gpt2")
def process(example):
    ids = enc.encode_ordinary(example['text']) # encode_ordinary ignores any special tokens
    ids.append(enc.eot_token) # add the end of text token, e.g. 50256 for gpt2 bpe
    # note: I think eot should be prepended not appended... hmm. it's called "eot" though...
    out = {'ids': ids, 'len': len(ids)}
    return out

In [10]:
process({'text':"Hello world"})

{'ids': [15496, 995, 50256], 'len': 3}

In [15]:
model = GPT.from_pretrained("gpt2", {})

loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 123.65M


In [23]:
wte = None
wpe = None
for name, param in model.named_parameters():
    if name == 'transformer.wte.weight':
        wte = param
    elif name == 'transformer.wpe.weight':
        wpe = param

In [22]:
c_attn = nn.Linear(c_attn_weight, bias=c_attn_bias)

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [24]:
wte_embed = nn.Embedding(50257, 768)
wpe_embed = nn.Embedding(50257, 768)
wte_embed.weight = wte
wpe_embed.weight = wpe

In [26]:
wte_embed(torch.tensor([15496, 995, 50256])).shape

torch.Size([3, 768])

In [ ]:
wte_OWT = wte_embed(torch.from_numpy((train_data).astype(np.int64)))

In [ ]:
wpe_OWT = wpe_embed(torch.from_numpy((train_data).astype(np.int64)))

In [ ]:
print(wte_OWT.shape, wpe_OWT.shape)